## **Phase 3: Data Preprocessing**

In [91]:
# Import Libraries
# libraries for reading and manipulating data
import pandas as pd
import numpy as np


In [92]:
# Import and Load our data
df = pd.read_csv("easy_visa_eda.csv")
df.head()

,case_id,continent,education_of_employee,has_job_experience,requires_job_training,no_of_employees,yr_of_estab,region_of_employment,prevailing_wage,unit_of_wage,full_time_position,case_status
0,EZYV01,Asia,High School,N,N,7227.0,2007.0,West,592.2029,Hour,Y,Denied
1,EZYV02,Asia,Master's,Y,N,2412.0,2002.0,Northeast,83425.6500,Year,Y,Certified
2,EZYV03,Asia,Bachelor's,N,Y,7227.0,2008.0,West,122996.8600,Year,Y,Denied
3,EZYV04,Asia,Bachelor's,N,N,98.0,1932.5,West,83434.0300,Year,Y,Denied
4,EZYV05,Africa,Master's,Y,N,1082.0,2005.0,South,149907.3900,Year,Y,Certified


In [93]:
df.set_index('case_id', inplace=True)

In [94]:
df.head()

,continent,education_of_employee,has_job_experience,requires_job_training,no_of_employees,yr_of_estab,region_of_employment,prevailing_wage,unit_of_wage,full_time_position,case_status
case_id,,,,,,,,,,,
EZYV01,Asia,High School,N,N,7227.0,2007.0,West,592.2029,Hour,Y,Denied
EZYV02,Asia,Master's,Y,N,2412.0,2002.0,Northeast,83425.6500,Year,Y,Certified
EZYV03,Asia,Bachelor's,N,Y,7227.0,2008.0,West,122996.8600,Year,Y,Denied
EZYV04,Asia,Bachelor's,N,N,98.0,1932.5,West,83434.0300,Year,Y,Denied
EZYV05,Africa,Master's,Y,N,1082.0,2005.0,South,149907.3900,Year,Y,Certified


### **Checking for missing values**

In [95]:
# We have checked for missing values during EDA but just to be thorough
def check_for_missing_values_and_duplicates(df):
    print("\n1. Missing Values:")
    missing_values = df.isnull().sum()
    if missing_values.sum() > 0:
        print(missing_values[missing_values > 0])
    else:
        print("No missing values found (as expected from EDA)")
    # 2. Check for duplicates
    print("\n2. Duplicate Rows:")
    duplicates = df.duplicated().sum()
    print(f"Number of duplicate rows: {duplicates}")
    if duplicates > 0:
        print(f"Percentage of duplicates: {(duplicates/len(df))*100:.2f}%")

check_for_missing_values_and_duplicates(df)


1. Missing Values:
No missing values found (as expected from EDA)

2. Duplicate Rows:
Number of duplicate rows: 0


### **Feature Engineering**

**Time-based Features**

In [96]:
# Getting the age of the company by deducting the year established from the current year

current_year = 2025

df['company_age'] = current_year - df['yr_of_estab']

df.head()

,continent,education_of_employee,has_job_experience,requires_job_training,no_of_employees,yr_of_estab,region_of_employment,prevailing_wage,unit_of_wage,full_time_position,case_status,company_age
case_id,,,,,,,,,,,,
EZYV01,Asia,High School,N,N,7227.0,2007.0,West,592.2029,Hour,Y,Denied,18.0
EZYV02,Asia,Master's,Y,N,2412.0,2002.0,Northeast,83425.6500,Year,Y,Certified,23.0
EZYV03,Asia,Bachelor's,N,Y,7227.0,2008.0,West,122996.8600,Year,Y,Denied,17.0
EZYV04,Asia,Bachelor's,N,N,98.0,1932.5,West,83434.0300,Year,Y,Denied,92.5
EZYV05,Africa,Master's,Y,N,1082.0,2005.0,South,149907.3900,Year,Y,Certified,20.0


**Standard Wage per Year**

In [97]:
# Getting a standard wage for every employee
hours_per_year = 2080
weeks_per_year = 52
months_per_year = 12

def standardize_wage(row):
    """Converts prevailing_wage to an annual standard based on the unit_of_wage."""
    unit = row['unit_of_wage']
    wage = row['prevailing_wage']

    if unit == 'Hour':
        return wage * hours_per_year
    elif unit == 'Week':
        return wage * weeks_per_year
    elif unit == 'Month':
        return wage * months_per_year
    elif unit == 'Year':
        return wage
    else:
        return np.nan

# Apply the function using the pandas 'apply' method
df['wage_per_year'] = df.apply(standardize_wage, axis=1)
df.head()

,continent,education_of_employee,has_job_experience,requires_job_training,no_of_employees,yr_of_estab,region_of_employment,prevailing_wage,unit_of_wage,full_time_position,case_status,company_age,wage_per_year
case_id,,,,,,,,,,,,,
EZYV01,Asia,High School,N,N,7227.0,2007.0,West,592.2029,Hour,Y,Denied,18.0,1231782.032
EZYV02,Asia,Master's,Y,N,2412.0,2002.0,Northeast,83425.6500,Year,Y,Certified,23.0,83425.650
EZYV03,Asia,Bachelor's,N,Y,7227.0,2008.0,West,122996.8600,Year,Y,Denied,17.0,122996.860
EZYV04,Asia,Bachelor's,N,N,98.0,1932.5,West,83434.0300,Year,Y,Denied,92.5,83434.030
EZYV05,Africa,Master's,Y,N,1082.0,2005.0,South,149907.3900,Year,Y,Certified,20.0,149907.390


**Deriving ratios from numerical features**

In [98]:
# getting the wage per year relative to the total number of employees

df["wage_employee_ratio"] = (df["wage_per_year"] / df["no_of_employees"]).round(2)

df.head()

,continent,education_of_employee,has_job_experience,requires_job_training,no_of_employees,yr_of_estab,region_of_employment,prevailing_wage,unit_of_wage,full_time_position,case_status,company_age,wage_per_year,wage_employee_ratio
case_id,,,,,,,,,,,,,,
EZYV01,Asia,High School,N,N,7227.0,2007.0,West,592.2029,Hour,Y,Denied,18.0,1231782.032,170.44
EZYV02,Asia,Master's,Y,N,2412.0,2002.0,Northeast,83425.6500,Year,Y,Certified,23.0,83425.650,34.59
EZYV03,Asia,Bachelor's,N,Y,7227.0,2008.0,West,122996.8600,Year,Y,Denied,17.0,122996.860,17.02
EZYV04,Asia,Bachelor's,N,N,98.0,1932.5,West,83434.0300,Year,Y,Denied,92.5,83434.030,851.37
EZYV05,Africa,Master's,Y,N,1082.0,2005.0,South,149907.3900,Year,Y,Certified,20.0,149907.390,138.55


In [99]:
# getting the rate of the employees growth per company
df['employees_growth_rate_ratio'] = (df['no_of_employees'] / df['company_age']).round(2)

df.head()

,continent,education_of_employee,has_job_experience,requires_job_training,no_of_employees,yr_of_estab,region_of_employment,prevailing_wage,unit_of_wage,full_time_position,case_status,company_age,wage_per_year,wage_employee_ratio,employees_growth_rate_ratio
case_id,,,,,,,,,,,,,,,
EZYV01,Asia,High School,N,N,7227.0,2007.0,West,592.2029,Hour,Y,Denied,18.0,1231782.032,170.44,401.50
EZYV02,Asia,Master's,Y,N,2412.0,2002.0,Northeast,83425.6500,Year,Y,Certified,23.0,83425.650,34.59,104.87
EZYV03,Asia,Bachelor's,N,Y,7227.0,2008.0,West,122996.8600,Year,Y,Denied,17.0,122996.860,17.02,425.12
EZYV04,Asia,Bachelor's,N,N,98.0,1932.5,West,83434.0300,Year,Y,Denied,92.5,83434.030,851.37,1.06
EZYV05,Africa,Master's,Y,N,1082.0,2005.0,South,149907.3900,Year,Y,Certified,20.0,149907.390,138.55,54.10


In [100]:
# Determining the wage per company age ratio.
df['wage_per_age_ratio'] = (df['wage_per_year'] / df['company_age']).round(2)
df.head()

,continent,education_of_employee,has_job_experience,requires_job_training,no_of_employees,yr_of_estab,region_of_employment,prevailing_wage,unit_of_wage,full_time_position,case_status,company_age,wage_per_year,wage_employee_ratio,employees_growth_rate_ratio,wage_per_age_ratio
case_id,,,,,,,,,,,,,,,,
EZYV01,Asia,High School,N,N,7227.0,2007.0,West,592.2029,Hour,Y,Denied,18.0,1231782.032,170.44,401.50,68432.34
EZYV02,Asia,Master's,Y,N,2412.0,2002.0,Northeast,83425.6500,Year,Y,Certified,23.0,83425.650,34.59,104.87,3627.20
EZYV03,Asia,Bachelor's,N,Y,7227.0,2008.0,West,122996.8600,Year,Y,Denied,17.0,122996.860,17.02,425.12,7235.11
EZYV04,Asia,Bachelor's,N,N,98.0,1932.5,West,83434.0300,Year,Y,Denied,92.5,83434.030,851.37,1.06,901.99
EZYV05,Africa,Master's,Y,N,1082.0,2005.0,South,149907.3900,Year,Y,Certified,20.0,149907.390,138.55,54.10,7495.37


In [101]:
df['unit_of_wage'].unique()

array(['Hour', 'Year', 'Week', 'Month'], dtype=object)

### **Encoding**

In [102]:
df.columns

Index(['continent', 'education_of_employee', 'has_job_experience',
       'requires_job_training', 'no_of_employees', 'yr_of_estab',
       'region_of_employment', 'prevailing_wage', 'unit_of_wage',
       'full_time_position', 'case_status', 'company_age', 'wage_per_year',
       'wage_employee_ratio', 'employees_growth_rate_ratio',
       'wage_per_age_ratio'],
      dtype='object')

In [103]:
df['yr_of_estab'].unique()

array([2007. , 2002. , 2008. , 1932.5, 2005. , 2012. , 1994. , 1995. ,
       2004. , 1963. , 2006. , 1987. , 1991. , 2001. , 1972. , 2013. ,
       1968. , 1981. , 1997. , 2009. , 1998. , 2000. , 2010. , 1965. ,
       2011. , 1989. , 1933. , 1960. , 2003. , 1976. , 1996. , 1935. ,
       1999. , 1947. , 1939. , 1970. , 1977. , 1982. , 1943. , 1956. ,
       1974. , 1985. , 1984. , 1971. , 1969. , 1988. , 1944. , 1975. ,
       1966. , 1993. , 1992. , 1979. , 1986. , 1962. , 1954. , 1946. ,
       1950. , 2014. , 1980. , 1949. , 1961. , 1951. , 1958. , 1983. ,
       1948. , 1945. , 1978. , 1967. , 2015. , 1938. , 1973. , 1959. ,
       1990. , 1940. , 1934. , 1952. , 1953. , 2016. , 1937. , 1942. ,
       1964. , 1936. , 1941. , 1957. , 1955. ])

In [104]:
# Create year of establishment bins for proper encoding
bins = [df['yr_of_estab'].min(), 1950, 1981, 2001, 2011, df['yr_of_estab'].max()]
labels = ['Pre-1950', '1950-1980', '1981-2000', '2001-2010', 'Post-2010']
df['establishment_period'] = pd.cut(
    df['yr_of_estab'],
    bins=bins,
    labels=labels,
    include_lowest=True
)
df = df.drop('yr_of_estab', axis=1)

df['establishment_period'].value_counts()

establishment_period
1981-2000    9221
2001-2010    7359
Pre-1950     4100
1950-1980    3513
Post-2010    1287
Name: count, dtype: int64

In [105]:
df['education_of_employee'].value_counts()

education_of_employee
Bachelor's     10234
Master's        9634
High School     3420
Doctorate       2192
Name: count, dtype: int64

In [106]:
df['continent'].value_counts()

continent
Asia             16861
Europe            3732
North America     3292
South America      852
Africa             551
Oceania            192
Name: count, dtype: int64

In [107]:
df['region_of_employment'].value_counts()

region_of_employment
Northeast    7195
South        7017
West         6586
Midwest      4307
Island        375
Name: count, dtype: int64

In [108]:
# Label Encoding for Case Status
# Encode the target for calculation: 1 for 'Certified', 0 for 'Denied'
df['case_status_encoded'] = df['case_status'].apply(lambda x: 1 if x == 'Certified' else 0)
df = df.drop('case_status', axis=1)

In [109]:
df['case_status_encoded'].value_counts()
df.head()

,continent,education_of_employee,has_job_experience,requires_job_training,no_of_employees,region_of_employment,prevailing_wage,unit_of_wage,full_time_position,company_age,wage_per_year,wage_employee_ratio,employees_growth_rate_ratio,wage_per_age_ratio,establishment_period,case_status_encoded
case_id,,,,,,,,,,,,,,,,
EZYV01,Asia,High School,N,N,7227.0,West,592.2029,Hour,Y,18.0,1231782.032,170.44,401.50,68432.34,2001-2010,0
EZYV02,Asia,Master's,Y,N,2412.0,Northeast,83425.6500,Year,Y,23.0,83425.650,34.59,104.87,3627.20,2001-2010,1
EZYV03,Asia,Bachelor's,N,Y,7227.0,West,122996.8600,Year,Y,17.0,122996.860,17.02,425.12,7235.11,2001-2010,0
EZYV04,Asia,Bachelor's,N,N,98.0,West,83434.0300,Year,Y,92.5,83434.030,851.37,1.06,901.99,Pre-1950,0
EZYV05,Africa,Master's,Y,N,1082.0,South,149907.3900,Year,Y,20.0,149907.390,138.55,54.10,7495.37,2001-2010,1


In [110]:
# Calculate the mean case status rate per region
region_cert_rate_map = df.groupby('region_of_employment')['case_status_encoded'].mean().round(2)
region_cert_rate_map

region_of_employment
Island       0.60
Midwest      0.76
Northeast    0.63
South        0.70
West         0.62
Name: case_status_encoded, dtype: float64

In [111]:
# Ordinal Encoding

# =================== Education of Employees using Ordinal Encoding ===========================
educational_mapping = {'High School': 1, 'Bachelor\'s': 2, 'Master\'s': 3, 'Doctorate': 4}

df['education_level_ordinal'] = df['education_of_employee'].map(educational_mapping)

df = df.drop('education_of_employee', axis=1)


# ====================== Establishment Year using Ordinal Encoding ==========================
establishment_period_mapping = {
    'Pre-1950': 5,
    '1950-1980': 4,
    '1981-2000': 3,
    '2001-2010': 2,
    'Post-2010': 1
}
df['establishment_period_ordinal'] = df['establishment_period'].map(establishment_period_mapping)
df = df.drop('establishment_period', axis=1)


# ======================= Continent using One Hot Encoding =========================
continent_dummies = pd.get_dummies(df['continent'], prefix='continent', drop_first=False).astype('int')
df = pd.concat([df, continent_dummies], axis=1)
df = df.drop('continent', axis=1)


# ==================== Binary Encoding for Y and N values ==========================
binary_map = {'Y': 1, 'N': 0}
df['has_job_experience_encoded'] = df['has_job_experience'].map(binary_map)
df['requires_job_training_encoded'] = df['requires_job_training'].map(binary_map)
df['full_time_position_encoded'] = df['full_time_position'].map(binary_map)
df = df.drop(['has_job_experience', 'requires_job_training', 'full_time_position'], axis=1)

# --- 5. Target Encoding: Region of Employment ---
# Calculate the mean certification rate per region
df['region_target_encoded'] = df['region_of_employment'].map(region_cert_rate_map)
df = df.drop('region_of_employment', axis=1)

df.head()

,no_of_employees,prevailing_wage,unit_of_wage,company_age,wage_per_year,wage_employee_ratio,employees_growth_rate_ratio,wage_per_age_ratio,case_status_encoded,education_level_ordinal,...,continent_Africa,continent_Asia,continent_Europe,continent_North America,continent_Oceania,continent_South America,has_job_experience_encoded,requires_job_training_encoded,full_time_position_encoded,region_target_encoded
case_id,,,,,,,,,,,,,,,,,,,,,
EZYV01,7227.0,592.2029,Hour,18.0,1231782.032,170.44,401.50,68432.34,0,1,...,0,1,0,0,0,0,0,0,1,0.62
EZYV02,2412.0,83425.6500,Year,23.0,83425.650,34.59,104.87,3627.20,1,3,...,0,1,0,0,0,0,1,0,1,0.63
EZYV03,7227.0,122996.8600,Year,17.0,122996.860,17.02,425.12,7235.11,0,2,...,0,1,0,0,0,0,0,1,1,0.62
EZYV04,98.0,83434.0300,Year,92.5,83434.030,851.37,1.06,901.99,0,2,...,0,1,0,0,0,0,0,0,1,0.62
EZYV05,1082.0,149907.3900,Year,20.0,149907.390,138.55,54.10,7495.37,1,3,...,1,0,0,0,0,0,1,0,1,0.70


### **Normalize using Log Transformation for skewed variables**

In [112]:
# Check skewness for variables identified in EDA as right-skewed

from scipy.stats import skew

def check_for_skewed_variables(df):
    print("\n3. Skewness Analysis (EDA identified right-skewed variables):")
    skewed_vars = ['prevailing_wage', 'no_of_employees']
    for var in skewed_vars:
        if var in df.columns:
            skewness = skew(df[var])
            print(f"{var}: skewness = {skewness:.3f} ({'right-skewed' if skewness > 0.5 else 'approximately normal'})")

check_for_skewed_variables(df)


3. Skewness Analysis (EDA identified right-skewed variables):
prevailing_wage: skewness = 0.547 (right-skewed)
no_of_employees: skewness = 0.959 (right-skewed)


In [113]:
# Log-transform skewed variables as recommended by EDA

print("=== LOG-TRANSFORMING SKEWED VARIABLES ===")
print("EDA identified these variables as right-skewed and recommended log transformation:")

# Variables to log-transform based on EDA findings
skewed_vars = ['prevailing_wage', 'no_of_employees']

for var in skewed_vars:
    if var in df.columns:
        # Check if variable has zero or negative values
        min_val = df[var].min()
        if min_val <= 0:
            # Use log1p for variables with zeros
            df[f'{var}_log'] = np.log1p(df[var])
            print(f"✓ {var}: Applied log1p transformation (had {min_val:.3f} minimum value)")
        else:
            # Use log for positive values only
            df[f'{var}_log'] = np.log(df[var])
            print(f"✓ {var}: Applied log transformation")
        
        # Check skewness before and after
        original_skew = skew(df[var])
        transformed_skew = skew(df[f'{var}_log'])
        print(f"  Original skewness: {original_skew:.3f} → Transformed skewness: {transformed_skew:.3f}")

print(f"\nDataset shape after log transformation: {df.shape}")
print("New log-transformed columns:", [col for col in df.columns if '_log' in col])

=== LOG-TRANSFORMING SKEWED VARIABLES ===
EDA identified these variables as right-skewed and recommended log transformation:
✓ prevailing_wage: Applied log transformation
  Original skewness: 0.547 → Transformed skewness: -2.142
✓ no_of_employees: Applied log1p transformation (had -26.000 minimum value)
  Original skewness: 0.959 → Transformed skewness: nan

Dataset shape after log transformation: (25480, 23)
New log-transformed columns: ['prevailing_wage_log', 'no_of_employees_log']


c:\Users\ncc666\Desktop\ai_engineering\module_three\my_venv\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


After log transformation, we have transformed skewness as nan

Hence, we need to investigate further

In [114]:
df[df['no_of_employees'] < 0]

,no_of_employees,prevailing_wage,unit_of_wage,company_age,wage_per_year,wage_employee_ratio,employees_growth_rate_ratio,wage_per_age_ratio,case_status_encoded,education_level_ordinal,...,continent_Europe,continent_North America,continent_Oceania,continent_South America,has_job_experience_encoded,requires_job_training_encoded,full_time_position_encoded,region_target_encoded,prevailing_wage_log,no_of_employees_log
case_id,,,,,,,,,,,,,,,,,,,,,
EZYV246,-25.0,39452.9900,Year,45.0,39452.990,-1578.12,-0.56,876.73,1,3,...,1,0,0,0,0,0,1,0.63,10.582865,NaN
EZYV379,-11.0,32506.1400,Year,14.0,32506.140,-2955.10,-0.79,2321.87,0,2,...,0,0,0,0,0,1,1,0.63,10.389184,NaN
EZYV833,-17.0,129701.9400,Year,23.0,129701.940,-7629.53,-0.74,5639.21,1,3,...,0,0,0,1,1,0,1,0.70,11.772994,NaN
EZYV2919,-26.0,112799.4600,Year,20.0,112799.460,-4338.44,-1.30,5639.97,1,3,...,0,0,0,0,1,0,1,0.76,11.633367,NaN
EZYV6440,-14.0,103.9700,Hour,12.0,216257.600,-15446.97,-1.17,18021.47,0,2,...,0,0,0,0,0,0,1,0.70,4.644102,NaN
EZYV6635,-26.0,5247.3200,Year,92.5,5247.320,-201.82,-0.28,56.73,0,2,...,0,0,0,0,1,0,1,0.62,8.565473,NaN
EZYV7225,-25.0,141435.9500,Year,27.0,141435.950,-5657.44,-0.93,5238.37,1,4,...,1,0,0,0,0,0,1,0.76,11.859602,NaN
EZYV7282,-14.0,58488.5000,Year,25.0,58488.500,-4177.75,-0.56,2339.54,0,1,...,0,0,0,0,0,0,1,0.76,10.976585,NaN
EZYV7319,-26.0,115005.6100,Year,19.0,115005.610,-4423.29,-1.37,6052.93,1,2,...,0,0,0,0,1,1,1,0.70,11.652736,NaN


We can see we have negative values for our number of employees columns which is invaluable, hence we drop them


In [115]:
df = df[df['no_of_employees'] > 0]
df[df['no_of_employees'] < 0]

,no_of_employees,prevailing_wage,unit_of_wage,company_age,wage_per_year,wage_employee_ratio,employees_growth_rate_ratio,wage_per_age_ratio,case_status_encoded,education_level_ordinal,...,continent_Europe,continent_North America,continent_Oceania,continent_South America,has_job_experience_encoded,requires_job_training_encoded,full_time_position_encoded,region_target_encoded,prevailing_wage_log,no_of_employees_log
case_id,,,,,,,,,,,,,,,,,,,,,


### **Feature Selection**

In [116]:
df.columns

Index(['no_of_employees', 'prevailing_wage', 'unit_of_wage', 'company_age',
       'wage_per_year', 'wage_employee_ratio', 'employees_growth_rate_ratio',
       'wage_per_age_ratio', 'case_status_encoded', 'education_level_ordinal',
       'establishment_period_ordinal', 'continent_Africa', 'continent_Asia',
       'continent_Europe', 'continent_North America', 'continent_Oceania',
       'continent_South America', 'has_job_experience_encoded',
       'requires_job_training_encoded', 'full_time_position_encoded',
       'region_target_encoded', 'prevailing_wage_log', 'no_of_employees_log'],
      dtype='object')

In [118]:
# Feature selection based on EDA correlation evidence
print("=== FEATURE SELECTION BASED ON EDA CORRELATION EVIDENCE ===")

# Separate features and target
X = df.drop('case_status_encoded', axis=1)
y = df['case_status_encoded']

print(f"Total features available: {X.shape[1]}")

# EDA-identified high-signal features (|correlation| > 0.2)
# There are no high-sgnal features with correlation > 0.2

# Engineered features 
engineered_features = ['company_age', 'wage_per_year']

# EDA-identified low-signal features (|correlation| < 0.1) 
low_signal_features = ['no_of_employees', 'prevailing_wage']
print(f"EDA-identified low-signal features: {low_signal_features}")

# Check which features are actually available
available_low_signal = [f for f in low_signal_features if f in X.columns]

print(f"Available low-signal features: {available_low_signal}")

# Create feature sets for evaluation
print("\n=== FEATURE SETS FOR EVALUATION ===")

# Set 1: All original features (excluding low-signal)
features_exclude_low_signal = [col for col in X.columns if col not in low_signal_features]
print(f"Set 1 - Exclude low-signal features: {len(features_exclude_low_signal)} features")

# Set 2: Engineered features
engineered = engineered_features + [col for col in X.columns if any(keyword in col.lower() for keyword in ['ratio', 'ordinal', 'encoded', 'continent', 'log'])]
print(f"Set 2 - Engineered: {len(engineered)} features")

# Set 3: All features (for comparison)
all_features = list(X.columns)
print(f"Set 3 - All features: {len(all_features)} features")

# Let's use Set 2 (high-signal + engineered) as recommended by EDA
selected_features = engineered
print(f"\n✓ Selected feature set: {len(selected_features)} features")
print("Selected features:", selected_features)


=== FEATURE SELECTION BASED ON EDA CORRELATION EVIDENCE ===
Total features available: 22
EDA-identified low-signal features: ['no_of_employees', 'prevailing_wage']
Available low-signal features: ['no_of_employees', 'prevailing_wage']

=== FEATURE SETS FOR EVALUATION ===
Set 1 - Exclude low-signal features: 20 features
Set 2 - Engineered: 19 features
Set 3 - All features: 22 features

✓ Selected feature set: 19 features
Selected features: ['company_age', 'wage_per_year', 'wage_employee_ratio', 'employees_growth_rate_ratio', 'wage_per_age_ratio', 'education_level_ordinal', 'establishment_period_ordinal', 'continent_Africa', 'continent_Asia', 'continent_Europe', 'continent_North America', 'continent_Oceania', 'continent_South America', 'has_job_experience_encoded', 'requires_job_training_encoded', 'full_time_position_encoded', 'region_target_encoded', 'prevailing_wage_log', 'no_of_employees_log']


In [119]:
# Save/Export preprocessed data
df.to_csv("preprocessed_easy_visa.csv", index=False)